In [28]:
import pandas as pd
import numpy as np
import tensorflow
import keras
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers import Dense, Flatten, Dropout , TimeDistributed, Input
from keras.optimizers import Adam

In [58]:
data=pd.read_csv('final.csv')

In [68]:
data.drop(columns=['Respondents','Labels','Unnamed: 0']).columns

Index(['Normalised_EAR', 'Normalised_MAR', 'Normalised_MOE',
       'Normalised_Circularity'],
      dtype='object')

In [59]:
z=5
x_train=np.array(data.drop(columns=['Respondents','Labels','Unnamed: 0']))
x_train=np.reshape(x_train,(-1,z,4))

In [60]:
y_train=np.array(data['Labels'])

In [61]:
y_shape=[]
for i in range(0,y_train.shape[0],z):
    y_shape.append(y_train[i])
y_shape=np.array(y_shape)

In [62]:
model=Sequential()
#model.add(Dense(1024,activation='sigmoid'))
model.add(LSTM(512,return_sequences=True,dropout=0.5,input_shape=(z,4)))
# model.add(TimeDistributed(Dense(1,'sigmoid')))
model.add(Flatten())
model.add(Dense(216,activation='sigmoid'))
model.add(Dense(32,activation='tanh'))
model.add(Dropout(0.5))
model.add(Dense(16,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer=Adam(lr=0.00005),loss='binary_crossentropy',metrics=['accuracy'])

In [63]:
hist=model.fit(x_train,y_shape,epochs=50,batch_size=10)

Epoch 1/50
36/36 [==============================] - 0s 6ms/step - loss: 0.8598 - accuracy: 0.6194
Epoch 2/50
36/36 [==============================] - 0s 5ms/step - loss: 0.7676 - accuracy: 0.6139
Epoch 3/50
36/36 [==============================] - 0s 5ms/step - loss: 0.7038 - accuracy: 0.6389
Epoch 4/50
36/36 [==============================] - 0s 5ms/step - loss: 0.6801 - accuracy: 0.6333
Epoch 5/50
36/36 [==============================] - 0s 5ms/step - loss: 0.6467 - accuracy: 0.6583
Epoch 6/50
36/36 [==============================] - 0s 5ms/step - loss: 0.6387 - accuracy: 0.6556
Epoch 7/50
36/36 [==============================] - 0s 5ms/step - loss: 0.5859 - accuracy: 0.6806
Epoch 8/50
36/36 [==============================] - 0s 6ms/step - loss: 0.6451 - accuracy: 0.6333
Epoch 9/50
36/36 [==============================] - 0s 5ms/step - loss: 0.6107 - accuracy: 0.6639
Epoch 10/50
36/36 [==============================] - 0s 5ms/step - loss: 0.5711 - accuracy: 0.7083
Epoch 11/50
36/36 [

In [66]:
model.save('LSTM2.h5',)

In [67]:
model.save_weights('weights.h5')

In [73]:
x_test=np.load('test.npy')
y_test=np.load('label_test.npy')
x_test=np.reshape(x_test,(-1,z,4))

In [74]:
model.predict(x_test)

array([[0.16856219],
       [0.16498724],
       [0.16498724],
       [0.16498724]], dtype=float32)

In [80]:
(model.predict(x_test)>0.5).astype('int')

array([[0],
       [0],
       [0],
       [0]])

In [81]:
m2=keras.models.load_model("LSTM")

In [83]:
m2.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_11 (LSTM)               (None, 5, 512)            1058816   
_________________________________________________________________
flatten_11 (Flatten)         (None, 2560)              0         
_________________________________________________________________
dense_48 (Dense)             (None, 216)               553176    
_________________________________________________________________
dense_49 (Dense)             (None, 32)                6944      
_________________________________________________________________
dropout_22 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_50 (Dense)             (None, 16)                528       
_________________________________________________________________
dropout_23 (Dropout)         (None, 16)              

In [86]:
a=m2.history